# MODNet (v0.1.10)

For now, this benchmark file simply loads our existing full benchmark results (265 MB) from [ml-evs/modnet-matbench](https://github.com/ml-evs/modnet-matbench) and exports them in the matbench format. Code for featurisation, hyperparameter optimisation and the final predictions themselves can be found in the aforementioned repository or in the illustrative run.py file.

In [2]:
def download_and_extract(url, fname):
    import urllib
    from zipfile import ZipFile
    import os
    if os.path.exists(fname):
        print(f"File {fname} already found, will not redownload.")
        return
    
    response = urllib.request.urlretrieve(url, fname)
    with ZipFile(fname, "r") as _zip:
        _zip.extractall(".")

# repo = "ml-evs/modnet-matbench"
repo = "ppdebreuck/modnet-matbench"

version = "main"
fname = f'modnet-matbench-{version.replace("#", "-").replace("/", "-")}'
if version.startswith("v"):
    url = f"https://github.com/{repo}/archive/refs/tags/{version}.zip"
else:
    url = f'https://github.com/{repo}/archive/refs/heads/{version.replace("#", "%23")}.zip'


download_and_extract(url, fname + ".zip")

In [3]:
import pickle
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

mb = MatbenchBenchmark(
    autoload=False, 
    subset=[
        'matbench_dielectric', 
        'matbench_jdft2d', 
        'matbench_steels', 
        'matbench_expt_gap', 
        'matbench_phonons',
        'matbench_log_gvrh',
        'matbench_log_kvrh',
        'matbench_glass', 
        'matbench_expt_is_metal',
        'matbench_perovskites',
        'matbench_mp_gap',
        'matbench_mp_is_metal',
        'matbench_mp_e_form'
    ],
)

results_locs = {task.dataset_name: f"{fname}/{task.dataset_name}/results/{task.dataset_name}_results.pkl" for task in mb.tasks}
# Remap filename for elastic tasks as they were joint-learned
results_locs["matbench_log_gvrh"] = results_locs["matbench_log_kvrh"] = f"{fname}/matbench_elastic/results/matbench_elastic_results.pkl"
target_key_map = {"matbench_log_gvrh": "log10G_VRH", "matbench_log_kvrh": "log10K_VRH"}

2021-10-08 10:30:29 INFO     Initialized benchmark 'matbench_v0.1' with 13 tasks: 
['matbench_dielectric',
 'matbench_jdft2d',
 'matbench_steels',
 'matbench_expt_gap',
 'matbench_phonons',
 'matbench_log_gvrh',
 'matbench_log_kvrh',
 'matbench_glass',
 'matbench_expt_is_metal',
 'matbench_perovskites',
 'matbench_mp_gap',
 'matbench_mp_is_metal',
 'matbench_mp_e_form']


In [4]:
for task in mb.tasks:
    task.load()
    with open(results_locs[task.dataset_name], "rb") as f:
        results = pickle.load(f)
        
    for fold_ind, fold in enumerate(task.folds):

        # Handle predictions that were made with joint/multitarget learning
        if task.dataset_name in target_key_map:
            predictions = results["predictions"][fold_ind][target_key_map[task.dataset_name]].values
        else:
            predictions = results["predictions"][fold_ind].values
        
        # Classification tasks must be recorded with labels and not group probabilities
        if task.metadata.task_type == CLF_KEY:
            predictions = predictions[:, 1] >= 0.5

        predictions = predictions.flatten()

        task.record(fold, predictions)
        
    if task.metadata.task_type == CLF_KEY:
        print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
        print(f"{task.dataset_name}: ROC score {task.scores['rocauc']['mean']}")
    else:
        print(f"{task.dataset_name}: MAE {task.scores['mae']['mean']}")

    task.df = None

2021-10-08 10:32:02 INFO     Loading dataset 'matbench_dielectric'...


Reading file /home/mevans/.local/conda/envs/modnet_matbench/lib/python3.8/site-packages/matminer/datasets/matbench_dielectric.json.gz: 4764it [00:06, 752.56it/s] 

2021-10-08 10:32:09 INFO     Dataset 'matbench_dielectric loaded.
2021-10-08 10:32:09 INFO     Recorded fold matbench_dielectric-0 successfully.
2021-10-08 10:32:09 INFO     Recorded fold matbench_dielectric-1 successfully.
2021-10-08 10:32:09 INFO     Recorded fold matbench_dielectric-2 successfully.
2021-10-08 10:32:09 INFO     Recorded fold matbench_dielectric-3 successfully.
2021-10-08 10:32:09 INFO     Recorded fold matbench_dielectric-4 successfully.


matbench_dielectric: MAE 0.2969698688737498
2021-10-08 10:32:09 INFO     Loading dataset 'matbench_jdft2d'...


Reading file /home/mevans/.local/conda/envs/modnet_matbench/lib/python3.8/site-packages/matminer/datasets/matbench_jdft2d.json.gz: 636it [00:00, 1782.27it/s]

...

2021-10-08 10:37:24 INFO     Recorded fold matbench_mp_is_metal-1 successfully.
2021-10-08 10:37:24 INFO     Recorded fold matbench_mp_is_metal-2 successfully.
2021-10-08 10:37:24 INFO     Recorded fold matbench_mp_is_metal-3 successfully.
2021-10-08 10:37:24 INFO     Recorded fold matbench_mp_is_metal-4 successfully.
matbench_mp_is_metal: Accuracy score 0.8030506180286311
matbench_mp_is_metal: ROC score 0.7804643191398983
2021-10-08 10:37:26 INFO     Loading dataset 'matbench_mp_e_form'...


Decoding objects from /home/mevans/.local/conda/envs/modnet_matbench/lib/python3.8/site-packages/matminer/datasets/matbench_mp_is_metal.json.gz: 100%|##########| 106113/106113 [02:33<00:00, 690.26it/s] 
Reading file /home/mevans/.local/conda/envs/modnet_matbench/lib/python3.8/site-packages/matminer/datasets/matbench_mp_e_form.json.gz: 132752it [02:59, 740.21it/s] 

2021-10-08 10:40:26 INFO     Dataset 'matbench_mp_e_form loaded.


2021-10-08 10:40:26 INFO     Recorded fold matbench_mp_e_form-0 successfully.
2021-10-08 10:40:26 INFO     Recorded fold matbench_mp_e_form-1 successfully.
2021-10-08 10:40:26 INFO     Recorded fold matbench_mp_e_form-2 successfully.
2021-10-08 10:40:26 INFO     Recorded fold matbench_mp_e_form-3 successfully.
2021-10-08 10:40:26 INFO     Recorded fold matbench_mp_e_form-4 successfully.
matbench_mp_e_form: MAE 0.044769163811452004


Decoding objects from /home/mevans/.local/conda/envs/modnet_matbench/lib/python3.8/site-packages/matminer/datasets/matbench_mp_e_form.json.gz: 100%|##########| 132752/132752 [03:09<00:00, 1081.19it/s]

In [5]:
mb.to_file("results.json.gz")

2021-10-08 10:58:16 INFO     Successfully wrote MatbenchBenchmark to file 'results.json.gz'.
